## **CLASE 3: Manejo profesional de errores globales y respuestas claras**

### Objetivos de aprendizaje:

* Centralizar y profesionalizar la gestión de errores.
* Responder con mensajes claros y seguros.
* Auditar errores en tiempo real con logging.

---


### Contenidos y desarrollo

#### 1. **Uso de `HTTPException` y respuestas estándar**



In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Optional

app = FastAPI()

# Define un modelo de datos para el recurso que se va a crear
class Recurso(BaseModel):
    nombre: str
    descripcion: Optional[str] = None # La descripción es opcional

# Simulación de una base de datos en memoria
# Almacenaremos los recursos con un ID único
db: List[Dict] = []
next_item_id = 1

# API inicial para mostrar algo en la web
@app.get("/")
def read_root():
    """
    Endpoint raíz que devuelve un mensaje de bienvenida.
    """
    return {"message": "¡Hola, mundo! Esta es tu API inicial."}

# Endpoint para obtener un recurso por su ID
@app.get("/recurso/{item_id}")
def read_item(item_id: int):
    if item_id < 0:
        raise HTTPException(status_code=422, detail="ID inválido")
    
    # Buscar el recurso en la "base de datos"
    for item in db:
        if item["item_id"] == item_id:
            return item
    
    raise HTTPException(status_code=404, detail="Recurso no encontrado")

# Nuevo endpoint GET para mostrar todos los registros
@app.get("/recursos/")
def get_all_recursos():
    """
    Recupera todos los recursos almacenados.
    """
    return {"recursos": db}

# Endpoint POST para agregar un nuevo registro
@app.post("/recurso/")
def create_item(recurso: Recurso):
    """
    Crea un nuevo recurso y lo agrega a la lista de registros.
    Asigna un ID único al nuevo recurso.
    """
    global next_item_id
    new_item = recurso.model_dump() # Convierte el modelo Pydantic a un diccionario
    new_item["item_id"] = next_item_id
    db.append(new_item)
    next_item_id += 1
    
    print(f"Recurso recibido y agregado: {new_item}")
    return {"message": "Recurso creado exitosamente", "data": new_item}

> Buenas prácticas:
>
> * Nunca exponer información interna en los `detail`.

---


#### 2. **Middleware de captura de errores globales**



In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse

app = FastAPI()

# Middleware global para capturar excepciones no controladas
@app.middleware("http")
async def catch_all_exceptions(request: Request, call_next):
    try:
        response = await call_next(request)
        return response
    except Exception as e:
        # Aquí podrías usar logging.error(str(e)) para guardar el error
        return JSONResponse(
            status_code=500,
            content={"detail": "Error interno del servidor"}
        )

# Ruta que funciona normalmente
@app.get("/")
async def read_root():
    return {"message": "Hola desde FastAPI"}

# Ruta que lanza una excepción intencional
@app.get("/error")
async def generate_error():
    raise ValueError("¡Esto es un error intencional!")



---

#### 3. **Logger básico con rotación de archivos**



In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
import logging

# Configuración del logging
logging.basicConfig(
    filename="api.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

app = FastAPI()


# Ruta principal
@app.get("/")
def root():
    return {"message": "Bienvenido a la API"}

# Ruta que genera un log informativo
@app.get("/log")
def log_demo():
    logging.info("Este es un log informativo desde /log")
    return {"message": "Log generado correctamente"}


---

#### 4. **Captura con bloques `try/except` y errores simulados**



In [ ]:
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import JSONResponse
import logging

# Configurar logging
logging.basicConfig(
    filename="api.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Bienvenido a la API"}

# Ruta que divide dos números recibidos por query params
@app.get("/divide")
def divide(x: int, y: int):
    try:
        resultado = x / y
        logging.info(f"División exitosa: {x} / {y} = {resultado}")
        return {"resultado": resultado}
    except ZeroDivisionError:
        logging.warning(f"Intento de división por cero: {x} / {y}")
        raise HTTPException(status_code=400, detail="No se puede dividir por cero")


### **Ejercicio:** Facturación

* `POST`: Crear una factura
* `GET /facturas`: Obtener todas las facturas
* `GET /facturas/{id}`: Obtener una factura por ID
* `PUT /facturas/{id}`: Actualizar una factura existente
* `DELETE /facturas/{id}`: Eliminar una factura
* Validaciones con `raise HTTPException`
* Middleware para errores inesperados
* Logging

---

### Escenario realista:

Desarrollar una API interna para la empresa que permita **gestionar facturas** de compras, calcular el precio unitario de productos, y manejar adecuadamente errores y trazabilidad.

---

#### Código  `main.py`



In [ ]:
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import logging

# Configurar logging
logging.basicConfig(
    filename="api.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

app = FastAPI(
    title="API Empresarial de Gestión de Facturas",
    description="Permite registrar y consultar facturas ingresadas manualmente.",
    version="1.0"
)

# Middleware para errores inesperados
@app.middleware("http")  # Middleware que intercepta todas las solicitudes HTTP antes de que lleguen a las rutas
async def catch_all_exceptions(request: Request, call_next):
    # request: representa la solicitud entrante
    # call_next: función que continúa el flujo y llama a la siguiente capa (por ejemplo, a la función de ruta)

    try:
        # Intentamos procesar la solicitud normalmente
        return await call_next(request)
    except Exception as e:
        # Si ocurre cualquier excepción no manejada, se captura aquí
        logging.error(f"Error interno: {str(e)}")  # Se registra el error en el archivo de logs
        # Se devuelve una respuesta controlada al cliente con estado 500 y un mensaje genérico
        return JSONResponse(status_code=500, content={"detail": "Error interno del servidor"})


# Modelo limpio sin restricciones automáticas
class Factura(BaseModel):
    cliente: str
    total: float
    cantidad: int

# Simulación de base de datos
facturas_db = {}
id_counter = 1

@app.get("/")
def root():
    return {"message": "API Empresarial de Gestión de Facturas"}

# POST - Crear factura (validación manual)
@app.post("/facturas", tags=["Facturas"], summary="Registrar nueva factura")
def crear_factura(factura: Factura):
    global id_counter

    # Validaciones manuales
    if factura.total < 0:
        raise HTTPException(status_code=400, detail="El total no puede ser negativo")
    if factura.cantidad <= 0:
        raise HTTPException(status_code=400, detail="La cantidad debe ser mayor que cero")

    factura_id = id_counter
    facturas_db[factura_id] = factura
    id_counter += 1

    precio_unitario = factura.total / factura.cantidad
    logging.info(f"Factura creada (ID {factura_id}) - Cliente: {factura.cliente}, Total: {factura.total}, Cantidad: {factura.cantidad}, Precio Unitario: {precio_unitario:.2f}")

    return {
        "id": factura_id,
        "cliente": factura.cliente,
        "precio_unitario": round(precio_unitario, 2)
    }

# GET todas las facturas
@app.get("/facturas", tags=["Facturas"])
def obtener_facturas():
    return {
        "total": len(facturas_db),
        "facturas": [
            {
                "id": fid,
                "cliente": f.cliente,
                "total": f.total,
                "cantidad": f.cantidad,
                "precio_unitario": round(f.total / f.cantidad, 2)
            } for fid, f in facturas_db.items()
        ]
    }

# GET por ID
@app.get("/facturas/{factura_id}", tags=["Facturas"])
def obtener_factura(factura_id: int):
    factura = facturas_db.get(factura_id)
    if not factura:
        raise HTTPException(status_code=404, detail="Factura no encontrada")
    return {
        "id": factura_id,
        "cliente": factura.cliente,
        "total": factura.total,
        "cantidad": factura.cantidad,
        "precio_unitario": round(factura.total / factura.cantidad, 2)
    }

# PUT - Actualizar
@app.put("/facturas/{factura_id}", tags=["Facturas"])
def actualizar_factura(factura_id: int, factura: Factura):
    if factura_id not in facturas_db:
        raise HTTPException(status_code=404, detail="Factura no encontrada")
    if factura.total < 0 or factura.cantidad <= 0:
        raise HTTPException(status_code=400, detail="Datos inválidos")
    facturas_db[factura_id] = factura
    return {"mensaje": f"Factura {factura_id} actualizada correctamente"}

# DELETE - Eliminar
@app.delete("/facturas/{factura_id}", tags=["Facturas"])
def eliminar_factura(factura_id: int):
    if factura_id not in facturas_db:
        raise HTTPException(status_code=404, detail="Factura no encontrada")
    del facturas_db[factura_id]
    return {"mensaje": f"Factura {factura_id} eliminada correctamente"}


---

## Cómo probar

1. Inicia con:

   ```bash
   uvicorn main:app --reload
   ```

2. Abre Swagger:
    [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

---
